In [18]:
import os
import random
import yfinance as yf
from PIL import Image, ImageDraw

# Отримання даних про акції та фінансову звітність
symbol = "COIN"
stock = yf.Ticker(symbol)
df = stock.history(period="6mo")

# Додаємо ковзні середні
df["SMA_50"] = df["Close"].rolling(window=50).mean()
df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()

# Отримуємо фінансову звітність
financials = stock.financials
balance_sheet = stock.balance_sheet
cashflow = stock.cashflow

# Фільтруємо можливі NaN
df.dropna(inplace=True)

# Беремо останнє значення
latest_close = df["Close"].iloc[-1]
latest_volume = df["Volume"].iloc[-1]
latest_sma = df["SMA_50"].iloc[-1]
latest_ema = df["EMA_20"].iloc[-1]

# Отримання додаткових фінансових параметрів
latest_net_income = financials.loc['Net Income'][0] if 'Net Income' in financials.index else 0
latest_assets = balance_sheet.loc['Total Assets'][0] if 'Total Assets' in balance_sheet.index else 0
latest_liabilities = balance_sheet.loc['Total Liab'][0] if 'Total Liab' in balance_sheet.index else 0
latest_operating_cashflow = cashflow.loc['Total Cash From Operating Activities'][0] if 'Total Cash From Operating Activities' in cashflow.index else 0
latest_revenue = financials.loc['Total Revenue'][0] if 'Total Revenue' in financials.index else 0
latest_market_cap = stock.info.get("marketCap", 1e9)  # Додаємо ринкову капіталізацію

# Функція масштабування
def scale_value(value, min_value, max_value, new_min, new_max):
    return new_min + (value - min_value) / (max_value - min_value) * (new_max - new_min)

# Визначення кольору неба
if latest_ema > 300:
    sky_color = (135, 206, 250)  # Блакитне небо
elif 150 < latest_ema <= 300:
    sky_color = (255, 255, 100)  # Жовте небо
elif 50 < latest_ema <= 150:
    sky_color = (128, 0, 128)  # Фіолетове небо
else:
    sky_color = (0, 0, 0)  # Чорне небо

# Створення зображення
img = Image.new("RGB", (800, 600), sky_color)
draw = ImageDraw.Draw(img)

# Лінія горизонту
horizon_y = int(600 * 0.6)

# Відділяємо небо і землю
draw.rectangle([0, horizon_y, 800, 600], fill=(60, 150, 60))

# Малювання гір
for i in range(4):
    gx = random.randint(50, 750)
    gy = horizon_y - scale_value(latest_assets, 1e9, 5e12, 100, 300)
    draw.polygon([(gx, gy), (gx - 100, horizon_y), (gx + 100, horizon_y)], fill=(80, 80, 80))

# Малювання будинків
building_x_positions = []
for i in range(5):
    x = 100 + i * 140
    y = horizon_y - scale_value(latest_assets, 1e9, 5e12, 50, 250)
    draw.rectangle([x, y, x + 80, horizon_y], fill=(120, 120, 120))
    building_x_positions.append(x)
    
    # Додаємо вікна залежно від доходу
    window_count = int(scale_value(latest_revenue, 1e9, 5e12, 5, 30))
    for w in range(window_count):
        wx = x + 10 + (w % 4) * 15
        wy = y + 10 + (w // 4) * 15
        if wy < horizon_y:
            draw.rectangle([wx, wy, wx + 10, wy + 10], fill=(255, 255, 0))

# Малювання вежі (відображає ринкову капіталізацію)
tower_x = max(building_x_positions) + 50
tower_y = horizon_y - scale_value(latest_market_cap, 1e9, 5e12, 100, 350)
draw.rectangle([tower_x - 20, tower_y, tower_x + 20, horizon_y], fill=(100, 100, 180), outline=(200, 200, 250), width=2)
draw.polygon([(tower_x - 20, tower_y), (tower_x + 20, tower_y), (tower_x, tower_y - 40)], fill=(150, 150, 200), outline=(255, 255, 255))

# Малювання дерев
tree_count = int(scale_value(latest_net_income, -1e9, 1e12, 5, 25))
for _ in range(tree_count):
    tx = random.randint(50, 750)
    ty = horizon_y - 10
    draw.rectangle([tx, ty, tx + 10, ty + 30], fill=(139, 69, 19))
    draw.ellipse([tx - 10, ty - 20, tx + 20, ty + 10], fill=(34, 139, 34))

# Малювання автомобілів
vehicle_count = int(scale_value(latest_operating_cashflow, -1e9, 1e12, 5, 40))
for _ in range(vehicle_count):
    vx = random.randint(50, 750)
    vy = horizon_y + 15
    draw.rectangle([vx, vy, vx + 30, vy + 15], fill=(255, 0, 0))

# Малювання птахів
bird_count = int(scale_value(latest_volume, 1e6, 5e9, 3, 15))
for _ in range(bird_count):
    bx = random.randint(50, 750)
    by = random.randint(50, 200)
    draw.text((bx, by), "V", fill=(0, 0, 0), anchor="mm")


# Збереження зображення
img_path = "/Users/mac/Documents/Univer/Crypto_lab/stock_city_nft3.png"
img.save(img_path)
img.show()


/var/folders/m6/cqv1zhq10ds_r6drrn1y2vzw0000gn/T/ipykernel_81133/793109382.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_net_income = financials.loc['Net Income'][0] if 'Net Income' in financials.index else 0
/var/folders/m6/cqv1zhq10ds_r6drrn1y2vzw0000gn/T/ipykernel_81133/793109382.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_assets = balance_sheet.loc['Total Assets'][0] if 'Total Assets' in balance_sheet.index else 0
/var/folders/m6/cqv1zhq10ds_r6drrn1y2vzw0000gn/T/ipykernel_81133/793109382.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ve